In [8]:
import csv
import tweepy
import spacy
import time
import pandas as pd

CONSUMER_KEY = 'u4dUGLKFa97pNzvP48CbcNK19'
CONSUMER_SECRET = 'eV7GXkhbA2gen2hcFpMM89oZGa5zql3WHeVGT5X4Tz3oXWSllF'

ACCESS_TOKEN = '2324788382-KMN2DDS8dWy6GW4GspZgvIDmZ8kuczkhFCa5Hwn'
ACCESS_SECRET = 'FegSIzzTDC09z1UG80iAVCleecX6aDJ4gnVJgw9zA8oGT'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

# Create the api to connect to twitter with your credentials
api = tweepy.API(auth, 
                 wait_on_rate_limit=True, 
                 wait_on_rate_limit_notify=True, 
                 compression=True)

In [9]:
# !python -m spacy download en_core_web_sm
# !python -m spacy download fr_core_news_sm

# import fr_core_news_sm
# import en_core_web_sm

# spacy_fr = fr_core_news_sm.load()
# spacy_en = en_core_web_sm.load()

In [10]:
# api.rate_limit_status()

In [11]:
def tokenize_fr(text):
    """
    Tokenizes French text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_fr.tokenizer(text)]

In [21]:
results_list = []
word_count = 0

# The search API allows for the collection of results from the last seven days.
for date in ['2020-02-28', '2020-02-29', '2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05']:
    
    print(date)
    
    # We will be searching for five companies in total.
    companies = ["Facebook", "Google", "Apple", "Amazon", "Microsoft"]
    
    i = 0
    
    for company in companies:
        
        search = api.search(company, count = 100, lang = "fr", exclude = "retweets", until = date)
        
        for tweet in search:
            tweet_dict = {}
            
            # filtering out quotes, verified users, and tweets that are too short (min length must be 5 words)
            # collecting tweet date, id, text, language, information about the user, and the company that it's referring to
            # additionally tracking the word count by taking the length of the tokenized tweet
            if not tweet._json['is_quote_status'] and not tweet._json["user"]["verified"] and len(tokenize_fr(tweet._json["text"])) > 4:
                tweet_dict['created_at'] = tweet._json["created_at"][4:10] + " " + tweet._json["created_at"][-4:]
                tweet_dict['tweet_id'] = tweet._json["id"]
                tweet_dict['text'] = tweet._json["text"]
                tweet_dict['lang'] = tweet._json["lang"] 
                tweet_dict['user_info'] = tweet._json["user"] 
                tweet_dict['company'] = companies[i]
                word_count += len(tokenize_fr(tweet._json["text"]))
            if len(tweet_dict) > 0: 
                results_list.append(tweet_dict)
        
        time.sleep(2)
        print("finished", companies[i], "sleeping now")
        
        i += 1
        
    time.sleep(5)
    print("going to next date")
    
print("Finished!\n")

print(word_count)
print(len(results_list))

2020-02-28
finished Facebook sleeping now
finished Google sleeping now
finished Apple sleeping now
finished Amazon sleeping now
finished Microsoft sleeping now
going to next date
2020-02-29
finished Facebook sleeping now
finished Google sleeping now
finished Apple sleeping now
finished Amazon sleeping now
finished Microsoft sleeping now
going to next date
2020-03-01
finished Facebook sleeping now
finished Google sleeping now
finished Apple sleeping now
finished Amazon sleeping now
finished Microsoft sleeping now
going to next date
2020-03-02
finished Facebook sleeping now
finished Google sleeping now
finished Apple sleeping now
finished Amazon sleeping now
finished Microsoft sleeping now
going to next date
2020-03-03
finished Facebook sleeping now
finished Google sleeping now
finished Apple sleeping now
finished Amazon sleeping now
finished Microsoft sleeping now
going to next date
2020-03-04
finished Facebook sleeping now
finished Google sleeping now
finished Apple sleeping now
finish

In [22]:
with open('twitter_french_results_last.csv', 'a', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['created_at', 'tweet_id', 'text', 'lang', 'user_info', 'company']
    writer = csv.DictWriter(csvfile, fieldnames = fieldnames)

    writer.writeheader() 
    for tweet in results_list:
        writer.writerow(tweet)

In [23]:
df = pd.read_csv('twitter_french_results_last.csv')
df.head()

,created_at,tweet_id,text,lang,user_info,company
0,Feb 27 2020,1233179891706757120,C'est sans commentaire. A vous de juger. Aim...,fr,"{'id': 1228803032835674112, 'id_str': '1228803...",Facebook
1,Feb 27 2020,1233179741131280384,Facebook ads faut leur dire que la hagra ça pa...,fr,"{'id': 700467253121310720, 'id_str': '70046725...",Facebook
2,Feb 27 2020,1233178953952776192,@wly_add Encore une légende Facebook obligé 🤣🤣,fr,"{'id': 1125160131736604679, 'id_str': '1125160...",Facebook
3,Feb 27 2020,1233178839280443392,L'être humain n'apprend que de ses propres exp...,fr,"{'id': 2459910193, 'id_str': '2459910193', 'na...",Facebook
4,Feb 27 2020,1233178802970406913,on dirait grave les arabes skinny tah 2009 qui...,fr,"{'id': 918025054570590208, 'id_str': '91802505...",Facebook


In [24]:
df.shape

(3161, 6)

In [25]:
df.groupby('company').describe(include = "all")

created_at                                                         \
               count unique          top freq mean  std  min  25%  50%  75%   
company                                                                       
Amazon           657      7  Mar 01 2020   96  NaN  NaN  NaN  NaN  NaN  NaN   
Apple            595      7  Mar 03 2020   90  NaN  NaN  NaN  NaN  NaN  NaN   
Facebook         625      7  Mar 01 2020   93  NaN  NaN  NaN  NaN  NaN  NaN   
Google           649      7  Feb 29 2020   96  NaN  NaN  NaN  NaN  NaN  NaN   
Microsoft        635      7  Feb 29 2020   95  NaN  NaN  NaN  NaN  NaN  NaN   

           ... user_info                                                     \
           ...    unique                                                top   
company    ...                                                                
Amazon     ...       444  {'id': 726499497522638850, 'id_str': '72649949...   
Apple      ...       491  {'id': 1495039002, 'id_str': '1495039002', 'na...   
Facebook   ...       565  {'id': 2322941492, 'id_str': '2322941492', 'na...   
Google     ...       564  {'id': 2427500503, 'id_str': '2427500503', 'na...   
Microsoft  ...       488  {'id': 1083899608101732353, 'id_str': '1083899...   

                                                   
          freq mean  std  min  25%  50%  75%  max  
company                                            
Amazon      65  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
Apple        8  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
Facebook     5  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
Google      20  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
Microsoft   16  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[5 rows x 55 columns]